In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [83]:
MOVIES_CSV_FILE = "ml-20m/movies.csv"

In [84]:
movies_rdd = sc.textFile(MOVIES_CSV_FILE)

In [85]:
movies_rdd = movies_rdd.filter(lambda x: x[0].isdigit())

In [86]:
movies_rdd.count()

27278

In [87]:
movies_rdd.take(10)

['1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy',
 '2,Jumanji (1995),Adventure|Children|Fantasy',
 '3,Grumpier Old Men (1995),Comedy|Romance',
 '4,Waiting to Exhale (1995),Comedy|Drama|Romance',
 '5,Father of the Bride Part II (1995),Comedy',
 '6,Heat (1995),Action|Crime|Thriller',
 '7,Sabrina (1995),Comedy|Romance',
 '8,Tom and Huck (1995),Adventure|Children',
 '9,Sudden Death (1995),Action',
 '10,GoldenEye (1995),Action|Adventure|Thriller']

Create 'movies_rdd' with each entry is a tuple (id, name, genres)

In [88]:
def split_total(x):
    a = x.split(',',1)
    
    b = a[1].rsplit(',',1)
    return [a[0]] + b

movies_rdd_tuple = movies_rdd.filter(lambda x: x[0].isdigit()).map(split_total)
movies_rdd_tuple.take(10)

[['1', 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'],
 ['2', 'Jumanji (1995)', 'Adventure|Children|Fantasy'],
 ['3', 'Grumpier Old Men (1995)', 'Comedy|Romance'],
 ['4', 'Waiting to Exhale (1995)', 'Comedy|Drama|Romance'],
 ['5', 'Father of the Bride Part II (1995)', 'Comedy'],
 ['6', 'Heat (1995)', 'Action|Crime|Thriller'],
 ['7', 'Sabrina (1995)', 'Comedy|Romance'],
 ['8', 'Tom and Huck (1995)', 'Adventure|Children'],
 ['9', 'Sudden Death (1995)', 'Action'],
 ['10', 'GoldenEye (1995)', 'Action|Adventure|Thriller']]

Create 'movies_title_rdd' with each entry is a tuple (id, name)

In [89]:
movies_title_rdd = movies_rdd_tuple.map(lambda x: [x[0]]+[x[1]])
movies_title_rdd.take(10)

[['1', 'Toy Story (1995)'],
 ['2', 'Jumanji (1995)'],
 ['3', 'Grumpier Old Men (1995)'],
 ['4', 'Waiting to Exhale (1995)'],
 ['5', 'Father of the Bride Part II (1995)'],
 ['6', 'Heat (1995)'],
 ['7', 'Sabrina (1995)'],
 ['8', 'Tom and Huck (1995)'],
 ['9', 'Sudden Death (1995)'],
 ['10', 'GoldenEye (1995)']]

Create 'movies_genres_rdd' with each entry is a tuple (id, genre) (a movie ID can appear several times)

In [90]:
def test (x):
    ID = x[0]
    #print(ID)
    genres = x[1]
    #print(genres)
    genres_split = genres.split('|')
    #print(genres_split)
    nb_genres = len(genres_split)
    nouvelle_liste = []
    for genre in genres_split:
        nouvelle_liste.append((ID,genre))
    return nouvelle_liste


movies_genre_rdd = movies_rdd_tuple.map(lambda x: [x[0]]+[x[2]])
movies_genre_rdd = movies_genre_rdd.flatMap(test)
movies_genre_rdd.take(10)

[('1', 'Adventure'),
 ('1', 'Animation'),
 ('1', 'Children'),
 ('1', 'Comedy'),
 ('1', 'Fantasy'),
 ('2', 'Adventure'),
 ('2', 'Children'),
 ('2', 'Fantasy'),
 ('3', 'Comedy'),
 ('3', 'Romance')]

List all the genres and count the number of times each genre appears

In [91]:
liste_genres = movies_genre_rdd.map(lambda x: x[1])
liste_genres.take(10)

['Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Fantasy',
 'Adventure',
 'Children',
 'Fantasy',
 'Comedy',
 'Romance']

In [92]:
nb_times_genre = liste_genres.map(lambda x:(x,1)).reduceByKey(lambda a,b:a+b)
nb_times_genre.take(10)

[('Children', 1139),
 ('Fantasy', 1412),
 ('Romance', 4127),
 ('Drama', 13344),
 ('Action', 3520),
 ('Thriller', 4178),
 ('Horror', 2611),
 ('Sci-Fi', 1743),
 ('IMAX', 196),
 ('Documentary', 2471)]

Create 'movie_year_rdd' with each entry is a tuple (id, year)

In [97]:
def split_id_year(x):
    a = x.split(',',1)
    print(a[1].split('(',1))
    b = a[1].split('(',1)
    date = b[1].split(')',1)[0]

    return [a[0]] + [date]

movie_year_rdd = movies_rdd.map(split_id_year)
movie_year_rdd.take(10)

[['1', '1995'],
 ['2', '1995'],
 ['3', '1995'],
 ['4', '1995'],
 ['5', '1995'],
 ['6', '1995'],
 ['7', '1995'],
 ['8', '1995'],
 ['9', '1995'],
 ['10', '1995']]

In [2]:
RATINGS_CSV_FILE = "ml-20m/ratings.csv"

In [3]:
ratings_rdd = sc.textFile(RATINGS_CSV_FILE)

In [4]:
ratings_rdd.take(1)

['userId,movieId,rating,timestamp']

In [5]:
ratings_rdd = ratings_rdd.filter(lambda x: x[0].isdigit())

In [6]:
ratings_rdd.take(30)

['1,2,3.5,1112486027',
 '1,29,3.5,1112484676',
 '1,32,3.5,1112484819',
 '1,47,3.5,1112484727',
 '1,50,3.5,1112484580',
 '1,112,3.5,1094785740',
 '1,151,4.0,1094785734',
 '1,223,4.0,1112485573',
 '1,253,4.0,1112484940',
 '1,260,4.0,1112484826',
 '1,293,4.0,1112484703',
 '1,296,4.0,1112484767',
 '1,318,4.0,1112484798',
 '1,337,3.5,1094785709',
 '1,367,3.5,1112485980',
 '1,541,4.0,1112484603',
 '1,589,3.5,1112485557',
 '1,593,3.5,1112484661',
 '1,653,3.0,1094785691',
 '1,919,3.5,1094785621',
 '1,924,3.5,1094785598',
 '1,1009,3.5,1112486013',
 '1,1036,4.0,1112485480',
 '1,1079,4.0,1094785665',
 '1,1080,3.5,1112485375',
 '1,1089,3.5,1112484669',
 '1,1090,4.0,1112485453',
 '1,1097,4.0,1112485701',
 '1,1136,3.5,1112484609',
 '1,1193,3.5,1112484690']

Create 'ratings_rdd' with each entry is a tuple (userId,movieId,rating,timestamp)

In [23]:
ratings_rdd_tuple = ratings_rdd.map(lambda x: x.split(',')).cache()
ratings_rdd_tuple.take(10)

[['1', '2', '3.5', '1112486027'],
 ['1', '29', '3.5', '1112484676'],
 ['1', '32', '3.5', '1112484819'],
 ['1', '47', '3.5', '1112484727'],
 ['1', '50', '3.5', '1112484580'],
 ['1', '112', '3.5', '1094785740'],
 ['1', '151', '4.0', '1094785734'],
 ['1', '223', '4.0', '1112485573'],
 ['1', '253', '4.0', '1112484940'],
 ['1', '260', '4.0', '1112484826']]

Compute the mean of all the notes using mean then meanApprox

In [8]:
ratings_rating = ratings_rdd_tuple.map(lambda x: x[2])
ratings_rating.take(10)

['3.5', '3.5', '3.5', '3.5', '3.5', '3.5', '4.0', '4.0', '4.0', '4.0']

In [9]:

ratings_mean = ratings_rating.map(lambda x : float(x)).mean()

In [10]:
ratings_mean

3.525528564299375

In [29]:
ratings_meanApprox = ratings_rating.map(lambda x : float(x)).meanApprox(timeout=1)

In [30]:
ratings_meanApprox

3.525528564299375

## Various way to group

Compute the mean of each movies using several methods

Using **groupByKey()**
* naive map-reduce approach: all keys are sent to reducers
* high network I/O (shuffle)

In [11]:
import numpy as np

In [14]:
ratings_gbkey = ratings_rdd_tuple.map(lambda x: (x[1],float(x[2]))).groupByKey().mapValues(lambda x: np.mean(list(x)))
ratings_gbkey.take(10)

[('50', 4.334372207803259),
 ('112', 3.412573591253154),
 ('1196', 4.188202061218635),
 ('2692', 4.025309765693817),
 ('2959', 4.227123123722136),
 ('3030', 4.211716774374825),
 ('3153', 3.601982097186701),
 ('4306', 3.843613161516327),
 ('5039', 3.312160694896851),
 ('5040', 2.8283082077051924)]

Using **reduceByKey()** 
* Warning function must be associative
* see shuffle read/write for the I/O gap

In [22]:
ratings_rbk = ratings_rdd_tuple.map(lambda x: (x[1],[float(x[2]),1]))\
                .reduceByKey(lambda a,b: [a[0]+b[0],a[1]+b[1]])\
                .mapValues(lambda x: x[0]/x[1])
ratings_rbk.take(10)

[('50', 4.334372207803259),
 ('112', 3.412573591253154),
 ('1196', 4.188202061218635),
 ('2692', 4.025309765693817),
 ('2959', 4.227123123722136),
 ('3030', 4.211716774374825),
 ('3153', 3.601982097186701),
 ('4306', 3.843613161516327),
 ('5039', 3.312160694896851),
 ('5040', 2.8283082077051924)]

Using **combineByKey()** ([ref to tutorial](https://github.com/mahmoudparsian/pyspark-tutorial/blob/master/tutorial/combine-by-key/spark-combineByKey.md))
* variant of reduceByKey, using init map and reduce lambdas
* equivalent to reduceByKey()

In [41]:
ratings_cbk = ratings_rdd_tuple.map(lambda x: (x[1],float(x[2]))).combineByKey(lambda value: (value, 1),
                            lambda x, value: (x[0] + value, x[1] + 1),
                            lambda x, y: (x[0] + y[0], x[1] + y[1])
                           ).mapValues(lambda x: x[0]/x[1])

ratings_cbk.take(10)

[('50', 4.334372207803259),
 ('112', 3.412573591253154),
 ('1196', 4.188202061218635),
 ('2692', 4.025309765693817),
 ('2959', 4.227123123722136),
 ('3030', 4.211716774374825),
 ('3153', 3.601982097186701),
 ('4306', 3.843613161516327),
 ('5039', 3.312160694896851),
 ('5040', 2.8283082077051924)]

Using **aggregateByKey()** ([ref to good doc](https://backtobazics.com/big-data/spark/apache-spark-aggregatebykey-example/))
* 3 parameters:
    * initial value for setting an accumulator for each key by partition
    * computer accumulator on each entry for each key of the partition
    * computer accumulators final value by key

In [49]:
def seq_op(accumulator, element):
    if(accumulator[1] > element[1]):
        return accumulator 
    else: 
        return element
 
 
# Combiner Operation : Finding Maximum Marks out Partition-Wise Accumulators
def comb_op(accumulator1, accumulator2):
    if(accumulator1[1] > accumulator2[1]):
        return accumulator1 
    else:
        return accumulator2
    
 
# Zero Value: Zero value in our case will be 0 as we are finding Maximum Marks
zero_val = (0, 0)
aggr_rdd = ratings_rdd_tuple.map(lambda t: (t[1], float(t[2])))\
                            .aggregateByKey((0,0),
                            lambda x, value: (x[0] + value, x[1] + 1),
                            lambda x, y: (x[0] + y[0], x[1] + y[1])
                           ).mapValues(lambda x: x[0]/x[1])
aggr_rdd.takeOrdered(10)

[('1', 3.921239561324077),
 ('10', 3.430029305292191),
 ('100', 3.22138517618469),
 ('1000', 3.105911330049261),
 ('100003', 3.6666666666666665),
 ('100006', 2.5),
 ('100008', 3.4166666666666665),
 ('100010', 2.7954545454545454),
 ('100013', 3.2),
 ('100015', 2.0)]

## Best movies

So what are the best ten movies (with more than 50 votes) ? 

In [ ]:
best_ten 